# Preparing Data for a Machine Learning Trading Strategy

## Background

Before adding the power of machine learning into a trading algorithm, it's crucial to prepare the data that you will use to fit the model.

In this activity, you’ll prepare training and testing data for fitting a machine learning-powered trading algorithm.

## Instructions

1. Read the provided OHLCV data provided in the CSV file into a Pandas DataFrame.

    > **Hint:** Remember to set the `date` columns as the DataFrame index and parse the dates.

2. Use the `pct_change` function to add a daily returns values column to the DataFrame. Name this column `actual_returns`.

    > **Hint:** Remove NAN values from the DataFrame.

3. Generate the features and target set as follows:

    * Set a short and long window size of 4 and 100 days, respectively, and add the fast and slow simple moving average columns to the DataFrame.

      > **Hint:** Remove NAN values from the DataFrame.

    * Create the features set by copying the `sma_fast` and `sma_slow` columns to a new DataFrame called `X`.

    * Add a `signal` column to the DataFrame setting its value to zeroes.

    * Use the Pandas `loc` function to populate the `signal` column as follows: where the `actual_returns` value is greater than or equal to zero, we set the `signal` value to 1. Where the `actual_returns` value is less than zero, we set the `signal` value to −1.

    * Create the target set `y` by copying the values of the `signal` column.

4. Split the data into training and testing sets as follows.

    * Use the pandas `DateOffset` module to set the beginning and end dates for the training the testing sets.

    * Set the `training_begin` date to the minimum date in the DataSet.

    * Set the ending period for the training data with an offset of 3 months

    * Use the `loc` function to generate the training datasets using the `training_begin` and `training_end` dates as lower and upper limits.

    * Create the testing sets using the `loc` function to slice the index starting at the `training_end` value and ending at the last record of the datasets.

5. Use the `StandardScaler` to standardize the training datasets.

In [1]:
# Imports
import pandas as pd
from pathlib import Path

## Read the CSV file into Pandas DataFrame

In [3]:
# Import the OHLCV dataset into a Pandas Dataframe

trading_df = pd.read_csv(
    Path("Resources/Week15-Day3-Actvity5-ohlcv.csv"), 
    index_col="date", 
    infer_datetime_format=True, 
    parse_dates=True
)

# Display sample data
display(trading_df.head())
display(trading_df.tail())

# Review the DataFrame
#trading_df.head()

,open,high,low,close,volume
date,,,,,
2018-10-19 09:30:00,16.90,17.18,16.90,17.095,11522
2018-10-19 09:45:00,17.11,17.44,17.11,17.400,70593
2018-10-19 10:00:00,17.40,17.40,17.25,17.280,38885
2018-10-19 10:15:00,17.27,17.27,17.18,17.200,37046
2018-10-19 10:30:00,17.21,17.37,17.19,17.200,46874


,open,high,low,close,volume
date,,,,,
2020-09-04 14:45:00,6.225,6.26,6.220,6.250,55512
2020-09-04 15:00:00,6.255,6.27,6.245,6.250,65810
2020-09-04 15:15:00,6.250,6.29,6.250,6.275,202630
2020-09-04 15:30:00,6.270,6.28,6.250,6.255,130140
2020-09-04 15:45:00,6.250,6.28,6.250,6.250,190278


## Add a Daily Return Values Column to the DataFrame

In [10]:
# Calculate the daily returns using the closing prices and the pct_change function
trading_df["actual_returns"] = trading_df["close"].pct_change()

# Drop all NaN values from the DataFrame
trading_df = trading_df.dropna()

# Review the DataFrame
display(trading_df.head())
display(trading_df.tail())

,open,high,low,close,volume,sma_fast,sma_slow,signal,actual_returns
date,,,,,,,,,
2018-10-24 15:00:00,15.66,15.66,15.600,15.615,80027,15.65250,16.3403,0.0,-0.002874
2018-10-24 15:15:00,15.63,15.66,15.520,15.530,76449,15.61875,16.3216,0.0,-0.005443
2018-10-24 15:30:00,15.54,15.54,15.180,15.410,137468,15.55375,16.3029,0.0,-0.007727
2018-10-24 15:45:00,15.41,15.42,15.350,15.350,688995,15.47625,16.2844,0.0,-0.003894
2018-10-25 09:30:00,15.55,15.55,15.205,15.320,44387,15.40250,16.2656,0.0,-0.001954


,open,high,low,close,volume,sma_fast,sma_slow,signal,actual_returns
date,,,,,,,,,
2020-09-04 14:45:00,6.225,6.26,6.220,6.250,55512,6.22875,6.27030,0.0,0.003210
2020-09-04 15:00:00,6.255,6.27,6.245,6.250,65810,6.23875,6.26985,0.0,0.000000
2020-09-04 15:15:00,6.250,6.29,6.250,6.275,202630,6.25125,6.26910,0.0,0.004000
2020-09-04 15:30:00,6.270,6.28,6.250,6.255,130140,6.25750,6.26855,0.0,-0.003187
2020-09-04 15:45:00,6.250,6.28,6.250,6.250,190278,6.25750,6.26785,0.0,-0.000799


## Generate the Features and Target Sets

### Add the Fast and Slow Simple Moving Average Columns to the DataFrame

In [11]:
# Define a window size of 4
short_window = 4

# Create a simple moving average (SMA) using the short_window and assign this to a new columns called sma_fast
trading_df["sma_fast"] = trading_df["close"].rolling(window=short_window).mean()

/Users/satishsurath/opt/anaconda3/envs/algotrading/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [12]:
# Define a window size of 100
long_window = 100

# Create an SMA that uses long_window, and assign it to a new columns named “sma_slow”
trading_df["sma_slow"] = trading_df["close"].rolling(window=long_window).mean()

/Users/satishsurath/opt/anaconda3/envs/algotrading/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [13]:
# Drop the NaNs using dropna()
trading_df = trading_df.dropna()
trading_df.head()

,open,high,low,close,volume,sma_fast,sma_slow,signal,actual_returns
date,,,,,,,,,
2018-10-30 13:45:00,14.775,14.815,14.760,14.79,44697,14.78625,15.29820,0.0,0.001015
2018-10-30 14:00:00,14.790,14.800,14.700,14.73,115450,14.76875,15.28935,0.0,-0.004057
2018-10-30 14:15:00,14.725,14.815,14.710,14.78,116849,14.76875,15.28185,0.0,0.003394
2018-10-30 14:30:00,14.780,14.800,14.755,14.79,49434,14.77250,15.27565,0.0,0.000677
2018-10-30 14:45:00,14.785,14.810,14.750,14.79,36988,14.77250,15.27005,0.0,0.000000


### Create the features set

In [14]:
# Assign a copy of the sma_fast and sma_slow columns to a new DataFrame called X
X = trading_df[["sma_fast", "sma_slow"]].shift().dropna().copy()

# Display sample data
display(X.head())
display(X.tail())

,sma_fast,sma_slow
date,,
2018-10-30 14:00:00,14.78625,15.29820
2018-10-30 14:15:00,14.76875,15.28935
2018-10-30 14:30:00,14.76875,15.28185
2018-10-30 14:45:00,14.77250,15.27565
2018-10-30 15:00:00,14.77250,15.27005


,sma_fast,sma_slow
date,,
2020-09-04 14:45:00,6.22625,6.27210
2020-09-04 15:00:00,6.22875,6.27030
2020-09-04 15:15:00,6.23875,6.26985
2020-09-04 15:30:00,6.25125,6.26910
2020-09-04 15:45:00,6.25750,6.26855


### Create the target set

In [15]:
# Create a new column in the trading_df called signal setting its value to zero.
trading_df["signal"] = 0.0

In [16]:
# Create the signal to buy
trading_df.loc[(trading_df["actual_returns"] >= 0), "signal"] = 1

In [17]:
# Create the signal to sell
trading_df.loc[(trading_df["actual_returns"] < 0), "signal"] = -1

In [18]:
# Copy the new signal column to a new Series called y.
y = trading_df["signal"].copy()

## Split the Data Into Training and Testing Datasets

### Set the Training Begin and End Dates

In [22]:
# Imports 
from pandas.tseries.offsets import DateOffset

In [23]:
# Select the start of the training period
training_begin =  X.index.min()

# Display the training begin date
print(training_begin)
print(X.index.max())

2018-10-30 14:00:00
2020-09-04 15:45:00


In [24]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=3)

# Display the training end date
print(training_end)

2019-01-30 14:00:00


### Create the Training Datasets

In [25]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Display sample data
X_train.head()

,sma_fast,sma_slow
date,,
2018-10-30 14:00:00,14.78625,15.29820
2018-10-30 14:15:00,14.76875,15.28935
2018-10-30 14:30:00,14.76875,15.28185
2018-10-30 14:45:00,14.77250,15.27565
2018-10-30 15:00:00,14.77250,15.27005


### Create the Testing Datasets

In [26]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Display sample data
X_test.head()

,sma_fast,sma_slow
date,,
2019-01-30 14:00:00,14.57875,14.30576
2019-01-30 14:15:00,14.57125,14.31006
2019-01-30 14:30:00,14.58500,14.31471
2019-01-30 14:45:00,14.61750,14.32026
2019-01-30 15:00:00,14.63375,14.32411


## Standardize the Data

In [27]:
# Imports
from sklearn.preprocessing import StandardScaler

In [28]:
# Create a StandardScaler instance
scaler = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)